In [5]:
import pandas as pd

In [7]:
from utils.archive2 import WGS84Coordinate
from utils.archive import MapboxApiClient, MapboxRoutingProfile, MapboxRouteEnum

In [10]:
df = pd.read_parquet("hilo_full_patterns.parquet")

In [12]:
# Extract latitude and longitude columns
valid_df = df[['latitude', 'longitude']].dropna()
coordinates = [
    WGS84Coordinate(lat=row['latitude'], lon=row['longitude'])
    for _, row in valid_df.iterrows()
]

In [13]:
# Create a walking routing profile
walking_profile = MapboxRoutingProfile(name=MapboxRouteEnum.WALKING)

# Initialize the Mapbox API client
api_client = MapboxApiClient()

In [14]:
# Generate the distance matrix
distance_matrix = api_client.make_distance_matrix(coordinates, walking_profile)

# Convert the matrix to a DataFrame
distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

print(distance_df)

  0%|          | 0/45290 [00:00<?, ?it/s]0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
  0%|          | 0/45290 [00:00<?, ?it/s]


RuntimeError: Error fetching distances from the Mapbox API. The server returned a "422-Unknown" status code with the text: "{"message":"Too many coordinates; maximum number of coordinates is 25.","code":"InvalidInput"}".